<a href="https://colab.research.google.com/github/anastasiaili/anastasia/blob/main/%D0%92%D0%BA%D1%83%D1%81%20%D0%BF%D1%80%D0%B8%D0%B1%D1%8B%D0%BB%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Проект "Вкус прибыли"**

У нас есть два файла:

1. bakeries.csv содержит информацию о некоторых торговых точках: id, название, адрес и стоимость ежемесячной аренды помещения
2. transactions.json содержит информацию о транзакциях, произведенных за 2024 г. в сети пекарен.

Ссылка, чтобы скачать файлы: https://disk.yandex.ru/d/Jm-TkQ3sMFngkw

О каждой транзакции известно следующее:

- transaction_id - уникальный идентификатор транзакции
- store_id - уникальный идентификатор торговой точки
- date - дата совершения транзакции
- total - сумма чека
- bonus - информация о программе лояльности. Если флаг receive установлен на true, то amount отражает количество начисленных клиенту бонусов. Иначе -- списание, в таком случае для подсчета полученной выручки списанные бонусы следует вычесть из суммы чека

P.S. Данные не основаны на реальной жизни, все совпадения случайны

**Загрузим и подготовим данные**

In [ ]:
import pandas as pd
import json
import pickle
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()
transactions = pd.read_json('transactions.json')

Saving transactions.json to transactions (5).json


In [ ]:
from google.colab import files
uploaded = files.upload()
bakeries = pd.read_csv('bakeries.csv')

Saving bakeries.csv to bakeries (1).csv


In [ ]:
with open('transactions.json', 'r') as f:
    transactions = json.load(f)
transaction_data = pd.json_normalize(transactions)
transaction_data['date'] = pd.to_datetime(transaction_data['date'])
transaction_data['month'] = transaction_data['date'].dt.month

#Crорректируем выручку с учетом бонусов
transaction_data['total'] = np.where(transaction_data['bonus.receive'] == False,
                                      transaction_data['total'] - transaction_data['bonus.amount'],
                                      transaction_data['total'])

#Сгруппируем данные по торговым точкам и месяцам с суммой выручки
monthly_revenue_by_store = transaction_data[['store_id', 'month', 'total']].groupby(['store_id', 'month']).sum()
sorted_monthly_revenue = monthly_revenue_by_store.sort_values(by=['month', 'store_id'], ascending=[True, True])  # Сортировка данных

#Сбросим индексы и сериализуем в pickle формат
monthly_revenue_data = sorted_monthly_revenue.reset_index()
with open('monthly_revenue_data.pkl', 'wb') as f:
    pickle.dump(monthly_revenue_data, f)

In [ ]:
monthly_revenue_data

,store_id,month,total
0,S-HSE_G123,1,32649
1,S-HSE_K3,1,37832
2,S-HSE_P17,1,37352
3,S-HSE_S55,1,42746
4,S-HSE_SP16,1,44567
...,...,...,...
67,S-HSE_K3,12,38362
68,S-HSE_P17,12,36934
69,S-HSE_S55,12,38932
70,S-HSE_SP16,12,44190


**Создадим словарь с теми точками, которые принесли максимальную прибыль за каждый месяц**

In [ ]:
max_revenue_stores = {}
for i in range(1, 13):
    monthly_data = transaction_data[transaction_data['month'] == i]
    max_store = monthly_data.loc[monthly_data['total'].idxmax()]
    monthly_name = pd.to_datetime(f'{i}-01', format='%m-%d').strftime('%B')
    max_revenue_stores[monthly_name] = max_store['store_id']

# Сохранение результатов в файл
with open('max_revenue_stores.pkl', 'wb') as f:
    pickle.dump(max_revenue_stores, f)

In [ ]:
max_revenue_stores

{'January': 'S-HSE_P17',
 'February': 'S-HSE_P17',
 'March': 'S-HSE_S55',
 'April': 'S-HSE_K3',
 'May': 'S-HSE_P17',
 'June': 'S-HSE_S55',
 'July': 'S-HSE_S55',
 'August': 'S-HSE_P17',
 'September': 'S-HSE_SP16',
 'October': 'S-HSE_K3',
 'November': 'S-HSE_G123',
 'December': 'S-HSE_SP16'}

**Объединим данные о пекарнях и выручке по месяца, учитывая только те торговые точки, информация по которым присутствует в обоих файлах**

In [ ]:
transaction_data = pd.DataFrame(transactions)

transaction_data['date'] = pd.to_datetime(transaction_data['date'])
transaction_data['month'] = transaction_data['date'].dt.month

#Смотрим, получен ли бонус
transaction_data['bonus.receive'] = transaction_data['bonus'].apply(lambda x: x['receive'])
transaction_data['bonus.amount'] = transaction_data['bonus'].apply(lambda x: x['amount'])

#Корректируем общую сумму транзакции, если бонус не был получен
transaction_data['total'] = np.where(transaction_data['bonus.receive'] == False, transaction_data['total'] - transaction_data['bonus.amount'], transaction_data['total'])

#Группируем данные по магазинам и мясяцам, суммируя соответствующие транзакции
monthly_revenue = (transaction_data.groupby(['store_id', 'month'])['total'].sum().unstack(fill_value=0).reset_index())
monthly_revenue.columns = ['id'] + ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

#объединяем данные, сортируем по id и сбрасываем индексы
merged_data = pd.merge(bakeries, monthly_revenue, on='id', how='inner')
merged_data = merged_data.sort_values(by='id').reset_index(drop=True)

In [ ]:
merged_data

,id,name,address,rent_fee,jan,feb,mar,apr,may,jun,...,apr_profit,may_profit,jun_profit,jul_profit,aug_profit,sep_profit,oct_profit,nov_profit,dec_profit,high_profit
0,S-HSE_G123,СЛОЙ Грибоедова,"наб. канала Грибоедова, 123",30000,32649,39198,44414,34240,47099,32386,...,4240,17099,2386,12110,8053,16687,1144,6753,10655,False
1,S-HSE_K3,СЛОЙ Кантемировская,"Кантемировская ул., 3, корп. 1",35000,37832,32854,41375,44408,41856,49337,...,9408,6856,14337,-2451,3025,17267,11903,4930,3362,False
2,S-HSE_P17,СЛОЙ Промышленная,"Промышленная ул., 17",26000,37352,36160,47881,35663,36614,33127,...,9663,10614,7127,22328,22415,8921,11734,21618,10934,True
3,S-HSE_S55,СЛОЙ Седова,"ул. Седова, 55, корп. 2",23000,42746,33578,46184,42575,38149,38044,...,19575,15149,15044,19501,20622,11656,19966,14194,15932,True
4,S-HSE_SP16,СЛОЙ Печатников,"ул. Союза Печатников, 16",28000,44567,34411,37942,32590,39688,31625,...,4590,11688,3625,10688,6631,12978,15975,6607,16190,False


**Вычислим точки, удержавшие прибыль на уровне больше 15% от выручки, и занесем их в список**

In [ ]:
list_of_months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

#Рассчитаем прибыль за каждый месяц как разницу между выручкой и стоимостью аренды (поскольку других затрат мы не знаем)
for month in list_of_months:
    merged_data[month + '_profit'] = merged_data[month] - merged_data['rent_fee']

#Создадим функцию для проверки, удерживает ли точка прибыль выше 15% от выручки каждый месяц
def check_high_profit(row):

    for month in list_of_months:
        revenue = row[month]
        profit = row[month+ '_profit']
        if profit / revenue <= 0.15:
            return False
    return True
#Применяем функцию к каждой строке DataFrame и создаем новый столбец
merged_data['high_profit'] = merged_data.apply(check_high_profit, axis=1)
#Формируем список названий торговых точек с высоким уровнем прибыли
successful_point_names = merged_data.loc[merged_data['high_profit'], 'name'].tolist()

In [ ]:
successful_point_names

['СЛОЙ Промышленная', 'СЛОЙ Седова']

# **A/B тестирование**

Узнаем, влияет ли изменение программы лояльности (разные условия списания и начисления бонусов) на общую выручку пекарен.

**H0:** Изменение условий начисления/списания бонусов не влияет на выручку.

**H1:** Изменение условий начисления/списания бонусов приводит к изменению выручки.

**Группа A (контрольная):** Клиенты, участвующие в текущей программе лояльности (существующие условия начисления и списания бонусов).

**Группа B (тестовая):** Клиенты, для которых изменены условия программы (например, увеличен процент начисляемых бонусов или изменены условия списания).


In [ ]:
from scipy.stats import ttest_ind

In [ ]:
transaction_data = pd.DataFrame(transactions)

transaction_data['date'] = pd.to_datetime(transaction_data['date'])
transaction_data['month'] = transaction_data['date'].dt.month
transaction_data['bonus.receive'] = transaction_data['bonus'].apply(lambda x: x['receive'])
transaction_data['bonus.amount'] = transaction_data['bonus'].apply(lambda x: x['amount'])
transaction_data['adjusted_total'] = np.where(transaction_data['bonus.receive'] == False,
                                               transaction_data['total'] - transaction_data['bonus.amount'],
                                               transaction_data['total'])

#Разделим на группы
np.random.seed(5000)
transaction_data['group'] = np.random.choice(['A', 'B'], size=len(transaction_data), p=[0.5, 0.5])

#Проанализируем средние чеки по группам
group_a = transaction_data[transaction_data['group'] == 'A']['adjusted_total']
group_b = transaction_data[transaction_data['group'] == 'B']['adjusted_total']
t_stat, p_value = ttest_ind(group_a, group_b, equal_var=False)

#Выведем результатов
print(f"Средний чек в группе A: {group_a.mean():.2f}")
print(f"Средний чек в группе B: {group_b.mean():.2f}")
print(f"T-статистика: {t_stat:.4f}, p-value: {p_value:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Отвергаем H0: программа лояльности повлияла на выручку")
else:
    print("Не удалось отвергнуть H0: нет статистически значимого влияния")

Средний чек в группе A: 575.36
Средний чек в группе B: 577.55
T-статистика: -0.3334, p-value: 0.7389
Не удалось отвергнуть H0: нет статистически значимого влияния


Мы видим, что p-value выше 0,05, значит на уровне значимости 5% у нас нет доказательств, что изменение программы лояльности влияет на средний чек, что может быть связано, например, с тем, что разница в начислении бонусов была небольшой, что покупатели не замечали разницы

# **Дашборд**
Чтобы визуализировать ключевые данные и наглядно представить результаты, создадим дашборд, где покажем:


*   Выручку по месяцам, чтобы отслеживать динамику доходов, выявить сезонные колебания
*   Средний чек для контрольной и тестовой группы, чтобы понять, как изменения в программе лояльности повлияли на поведение клиентов
*   Топ-3 прибыльных точки, чтобы в дальнейшем оптимизировать операции, улучшать маркетинг и перераспределять ресурсы

In [ ]:
import plotly.express as px

In [ ]:
transaction_data['date'] = pd.to_datetime(transaction_data['date'])
transaction_data['month'] = transaction_data['date'].dt.strftime('%b')

#Выручка по месяцам
monthly_revenue = transaction_data.groupby('month')['total'].sum().reset_index()

#Средний чек по группам
grouped_data = transaction_data.groupby('group')['total'].mean().reset_index()

#График выручки по месяцам
fig_revenue = px.line(monthly_revenue, x='month', y='total', title="Выручка по месяцам")

#Гистограмма среднего чека в группах A/B
fig_ab_test = px.bar(grouped_data, x='group', y='total', title="Средний чек в группах A и B")

#Топ-3 прибыльных пекарни
top_bakeries = transaction_data.groupby('store_id')['total'].sum().nlargest(3).reset_index()

#Выводим графики
print("Выручка по месяцам:")
fig_revenue.show()

print("A/B-тест: сравнение среднего чека")
fig_ab_test.show()

print("🏆 Топ-3 прибыльных пекарен:")
print(top_bakeries[['store_id', 'total']])

Выручка по месяцам:


A/B-тест: сравнение среднего чека


🏆 Топ-3 прибыльных пекарен:
    store_id   total
0   S-SPb_M3  524232
1   S-HSE_K3  517113
2  S-HSE_S55  503892
